In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore')

In [2]:
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    model = xgb.XGBClassifier(eval_metric='error')
    model.fit(train_X, train_y)

    # y_pred = model.predict(validate_X)
    # print(cross_val_score(validate_y, y_pred, cv=5))

    question_accuracy = np.mean(cross_val_score(model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.6774901891915623
Question 2 accuracy: 0.980691788640401
Question 3 accuracy: 0.9363464842070188
Question 4 accuracy: 0.7865487793620666
Question 5 accuracy: 0.5327763180705748
Question 6 accuracy: 0.7475068276022
Question 7 accuracy: 0.6942465771929943
Question 8 accuracy: 0.555272394872938
Question 9 accuracy: 0.7029496592390461
Question 10 accuracy: 0.4977737401300903
Question 11 accuracy: 0.5593038885249001
Question 12 accuracy: 0.8533847570544385
Question 13 accuracy: 0.6889441251100409
Question 14 accuracy: 0.66581966124286
Question 15 accuracy: 0.5064655647941136
Question 16 accuracy: 0.6934002472121094
Question 17 accuracy: 0.6535079128138277
Question 18 accuracy: 0.9473796191852808
Overall accuracy: 0.704433807469248


In [4]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    param_dist = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(1, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'criterion': ['gini', 'entropy'],
    }

    base_model = RandomForestClassifier()

    random_search = RandomizedSearchCV(base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)

    random_search.fit(train_X, train_y)

    best_params = random_search.best_params_
    print(f"Best hyperparameters for question {q_no}:", best_params)

    best_model = RandomForestClassifier(**best_params)
    best_model.fit(train_X, train_y)

    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Best hyperparameters for question 1: {'criterion': 'entropy', 'max_depth': 42, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 342}
Question 1 accuracy: 0.7237429444358139
Best hyperparameters for question 2: {'criterion': 'gini', 'max_depth': 33, 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 333}
Question 2 accuracy: 0.9828135800050883
Best hyperparameters for question 3: {'criterion': 'entropy', 'max_depth': 21, 'min_samples_leaf': 15, 'min_samples_split': 15, 'n_estimators': 278}
Question 3 accuracy: 0.9323149905550565
Best hyperparameters for question 4: {'criterion': 'entropy', 'max_depth': 38, 'min_samples_leaf': 13, 'min_samples_split': 13, 'n_estimators': 304}
Question 4 accuracy: 0.7984298203546976
Best hyperparameters for question 5: {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 6, 'min_samples_split': 15, 'n_estimators': 391}
Question 5 accuracy: 0.5533629177333035
Best hyperparameters for question 6: {'criterion': 'gini', 'max_de

In [11]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']


    best_model = RandomForestClassifier()
    best_model.fit(train_X, train_y)

    question_accuracy = np.mean(cross_val_score(best_model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.7311687639169385
Question 2 accuracy: 0.9800552962605229
Question 3 accuracy: 0.9310417806476596
Question 4 accuracy: 0.7950341436396917
Question 5 accuracy: 0.5043525541874084
Question 6 accuracy: 0.7702100402338834
Question 7 accuracy: 0.7315945181052474
Question 8 accuracy: 0.5964390649168193
Question 9 accuracy: 0.7239548083655857
Question 10 accuracy: 0.4839897512794014
Question 11 accuracy: 0.6276280921214086
Question 12 accuracy: 0.8629323679002505
Question 13 accuracy: 0.7188630944742014
Question 14 accuracy: 0.6832154685434974
Question 15 accuracy: 0.5069021260691698
Question 16 accuracy: 0.7303199573120074
Question 17 accuracy: 0.6793974148547911
Question 18 accuracy: 0.9509867095347774
Overall accuracy: 0.722671441797959


In [13]:
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

# Assuming you have already read and preprocessed the data
train = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

# Specify the categorical columns to be used by the CatBoost model
cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

# Hyperparameter tuning configuration
param_dist = {
    'iterations': [100, 300, 500, 700],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'depth': [3, 4, 6, 8],
    'verbose': [0]
}

# Loop through questions, create train & validation sets for each question
for q_no in range(1, 19):

    # Filter dataset based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    # Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Hyperparameter tuning for the CatBoost model
    catboost_base_model = CatBoostClassifier(cat_features=cat_features)
    catboost_random_search = RandomizedSearchCV(catboost_base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
    catboost_random_search.fit(train_X, train_y)
    best_catboost_params = catboost_random_search.best_params_

    # Train the CatBoost model with the best hyperparameters
    model = CatBoostClassifier(cat_features=cat_features, **best_catboost_params)
    model.fit(train_X, train_y)

    # Calculate the accuracy using cross_val_score
    question_accuracy = np.mean(cross_val_score(model, validate_X, validate_y, cv=5))
    print("Question", q_no, "accuracy:", question_accuracy)
    total_accuracy += question_accuracy

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.743899962400344


In [12]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
warnings.simplefilter(action='ignore')

# Assuming you have already read and preprocessed the data
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

total_accuracy = 0

# Hyperparameter tuning configuration
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [-1, 3, 5, 7, 9],
    'random_state': [42]
}

# Loop through questions, create train & validation sets for each question
for q_no in range(1, 19):

    # Filter dataset based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    # Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Hyperparameter tuning for the LightGBM model
    lgbm_base_model = LGBMClassifier()
    lgbm_random_search = RandomizedSearchCV(lgbm_base_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
    lgbm_random_search.fit(train_X, train_y)
    best_lgbm_params = lgbm_random_search.best_params_

    # Train the LightGBM model with the best hyperparameters
    model = LGBMClassifier(**best_lgbm_params)
    model.fit(train_X, train_y)

    # Calculate the accuracy using cross_val_score
    question_accuracy = np.mean(cross_val_score(model, validate_X, validate_y, cv=5))
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.7241671225906389
Question 2 accuracy: 0.9762361168336137
Question 3 accuracy: 0.9327391687098816
Question 4 accuracy: 0.8058560901311035
Question 5 accuracy: 0.5588801606653563
Question 6 accuracy: 0.7729683239784488
Question 7 accuracy: 0.7396568299662504
Question 8 accuracy: 0.6216844195581253
Question 9 accuracy: 0.7356248860190069
Question 10 accuracy: 0.49076894673682264
Question 11 accuracy: 0.6475705443844801
Question 12 accuracy: 0.8650541592649379
Question 13 accuracy: 0.7281986162426011
Question 14 accuracy: 0.706131895990796
Question 15 accuracy: 0.5228112834991545
Question 16 accuracy: 0.7256528718707292
Question 17 accuracy: 0.6789730115523254
Question 18 accuracy: 0.9548061141093273
Overall accuracy: 0.732654475672422


In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
warnings.simplefilter(action='ignore')

train_other = pd.read_csv('StudentPerformancePred.csv')
train_catboost = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]))
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]))

train_other['session_id'] = train_other['session_id'].astype(str)
train_catboost['session_id'] = train_catboost['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

total_accuracy = 0

for q_no in range(1, 19):
    if q_no <= 3:
        select_other = train_other[train_other['level_group'] == '0-4']
        select_catboost = train_catboost[train_catboost['level_group'] == '0-4']
    elif q_no <= 13:
        select_other = train_other[train_other['level_group'] == '5-12']
        select_catboost = train_catboost[train_catboost['level_group'] == '5-12']
    else:
        select_other = train_other[train_other['level_group'] == '13-22']
        select_catboost = train_catboost[train_catboost['level_group'] == '13-22']

    train_X_other, validate_X_other = train_test_split(select_other, test_size=0.2)
    train_X_catboost, validate_X_catboost = train_test_split(select_catboost, test_size=0.2)

    train_y = labels.loc[(labels['session'].isin(train_X_other['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X_other['session_id'])) & (labels['q'] == q_no)]

    train_X_other = train_X_other.drop(['session_id', 'level_group'], axis=1)
    train_X_catboost = train_X_catboost.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X_other = validate_X_other.drop(['session_id', 'level_group'], axis=1)
    validate_X_catboost = validate_X_catboost.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    # Train the four models
    rf_model = RandomForestClassifier(n_estimators=100)
    xgb_model = XGBClassifier()
    cat_model = CatBoostClassifier(cat_features=cat_features, iterations=500, learning_rate=0.1, depth=6, verbose=0)
    lgbm_model = LGBMClassifier()

    # Create the voting classifier
    voting_classifier = VotingClassifier(
        estimators=[('rf', rf_model), ('xgb', xgb_model), ('cat', cat_model), ('lgbm', lgbm_model)],
        voting='soft'
    )

    rf_model.fit(train_X_other, train_y)
    xgb_model.fit(train_X_other, train_y)
    cat_model.fit(train_X_catboost, train_y)
    lgbm_model.fit(train_X_other, train_y)

    # Obtain the predictions
    rf_pred = rf_model.predict_proba(validate_X_other)[:, 1]
    xgb_pred = xgb_model.predict_proba(validate_X_other)[:, 1]
    cat_pred = cat_model.predict_proba(validate_X_catboost)[:, 1]
    lgbm_pred = lgbm_model.predict_proba(validate_X_other)[:, 1]

    # Average the predictions
    ensemble_pred = (rf_pred + xgb_pred + cat_pred + lgbm_pred) / 4
    ensemble_pred = np.round(ensemble_pred).astype(int)

    # Calculate the accuracy of the ensemble model
    question_accuracy = np.mean(ensemble_pred == validate_y)
    total_accuracy += question_accuracy
    print("Question", q_no, "accuracy:", question_accuracy)

overall_accuracy = total_accuracy / 18
print("Overall accuracy:", overall_accuracy)


Question 1 accuracy: 0.7165287502652239
Question 2 accuracy: 0.9777211966900063
Question 3 accuracy: 0.9295565457245916
Question 4 accuracy: 0.80500742626777
Question 5 accuracy: 0.539147040101846
Question 6 accuracy: 0.7644812221514958
Question 7 accuracy: 0.7358370464672184
Question 8 accuracy: 0.6040738383195416
Question 9 accuracy: 0.7265011669849353
Question 10 accuracy: 0.5128368342881392
Question 11 accuracy: 0.6346276257161044
Question 12 accuracy: 0.8663271801400382
Question 13 accuracy: 0.7224697644812221
Question 14 accuracy: 0.7008274984086569
Question 15 accuracy: 0.4988330150647146
Question 16 accuracy: 0.7388075535752174
Question 17 accuracy: 0.6893698281349459
Question 18 accuracy: 0.954169318905156
Overall accuracy: 0.7287290473159348


In [4]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

warnings.simplefilter(action='ignore')

train = pd.read_csv('StudentPerformancePred.csv')
train_catboost = pd.read_csv('CatBoostData.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]))
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]))

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

cat_features = ['event_name_mode', 'fqid_mode', 'room_fqid_mode', 'text_mode']

total_accuracy = 0

for q_no in range(1, 19):
    # Select train data based on current question
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
        select_catboost = train_catboost[train_catboost['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
        select_catboost = train_catboost[train_catboost['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']
        select_catboost = train_catboost[train_catboost['level_group'] == '13-22']

    # Create train & validation sets
    train_X_other, validate_X_other = train_test_split(select, test_size=0.2)
    train_X_catboost, validate_X_catboost = train_test_split(select_catboost, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X_other['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X_other['session_id'])) & (labels['q'] == q_no)]

    train_X_other = train_X_other.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X_other = validate_X_other.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    train_X_catboost = train_X_catboost.drop(['session_id', 'level_group'], axis=1)
    validate_X_catboost = validate_X_catboost.drop(['session_id', 'level_group'], axis=1)
# Define hyperparameter distributions for each model
rf_param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'criterion': ['gini', 'entropy'],
}

xgb_param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 50),
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'min_child_weight': randint(1, 10),
    'gamma': np.linspace(0, 1, 10),
}

lgbm_param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 50),
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'num_leaves': randint(20, 200),
    'min_child_samples': randint(1, 100),
}

catboost_param_dist = {
    'iterations': randint(100, 500),
    'depth': randint(1, 16),
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'l2_leaf_reg': randint(2, 30),
    'border_count': randint(1, 255),
}

# Perform hyperparameter tuning for each model
rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=rf_param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
rf_random_search.fit(train_X_other, train_y)
best_rf_params = rf_random_search.best_params_

xgb_random_search = RandomizedSearchCV(XGBClassifier(), param_distributions=xgb_param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
xgb_random_search.fit(train_X_other, train_y)
best_xgb_params = xgb_random_search.best_params_

lgbm_random_search = RandomizedSearchCV(LGBMClassifier(), param_distributions=lgbm_param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
lgbm_random_search.fit(train_X_other, train_y)
best_lgbm_params = lgbm_random_search.best_params_

catboost_random_search = RandomizedSearchCV(CatBoostClassifier(cat_features=cat_features, verbose=0), param_distributions=catboost_param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
catboost_random_search.fit(train_X_catboost, train_y)
best_catboost_params = catboost_random_search.best_params_

# Train the models with the best hyperparameters
rf_model = RandomForestClassifier(**best_rf_params)
rf_model.fit(train_X_other, train_y)

xgb_model = XGBClassifier(**best_xgb_params)
xgb_model.fit(train_X_other, train_y)

lgbm_model = LGBMClassifier(**best_lgbm_params)
lgbm_model.fit(train_X_other, train_y)
   
# Train the CatBoost model
catboost_model = CatBoostClassifier(cat_features=cat_features, verbose=0, **best_catboost_params)
catboost_model.fit(train_X_catboost, train_y)

# Create the ensemble model with weights
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('catboost', catboost_model),
    ],
    voting='soft',
    weights=[3, 1, 1, 1],  # Higher weight for the random forest model
)

# Assuming train_X_other and validate_X_other have the same features
train_X_catboost = train_X_catboost[train_X_other.columns]
validate_X_catboost = validate_X_catboost[train_X_other.columns]


# Fit the ensemble model
ensemble_model.fit(train_X_other, train_y)

 # Calculate the accuracy using cross_val_score
accuracy = np.mean(cross_val_score(ensemble_model, validate_X_other, validate_y, cv=5))
total_accuracy += accuracy
print("Question", q_no, "accuracy:", accuracy)

print("Total accuracy:", total_accuracy / 18)

KeyboardInterrupt: 